## Shuffle in the Cross-decoder

### Load all dictionaries and Data

In [6]:
# Load all dictionaries
#%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf


##if you are the local
#os.chdir('C:\\Users\\David\\Dropbox\\IDIBAPS\\Distractor_neurons\\FR_trials') #David_all_cond

## if your are in the cluster
os.chdir('/home/david/Desktop/IDIBAPS/Gottlib/FR_trials')



##Load the data
### Target
LIP_control = np.load('control_LIP.npy')
LIP_0 = np.load('0_LIP.npy')
LIP_100 = np.load('100_LIP.npy')
LIP_200 = np.load('200_LIP.npy')
LIP_800 = np.load('800_LIP.npy')
####
DLPFC_control = np.load('control_DLPFC.npy')
DLPFC_0 = np.load('0_DLPFC.npy')
DLPFC_100 = np.load('100_DLPFC.npy')
DLPFC_200 = np.load('200_DLPFC.npy')
DLPFC_800 = np.load('800_DLPFC.npy')

### Distractor
LIP_control_d = np.load('control_LIP_d.npy')
LIP_0_d = np.load('0_LIP_d.npy')
LIP_100_d = np.load('100_LIP_d.npy')
LIP_200_d = np.load('200_LIP_d.npy')
LIP_800_d = np.load('800_LIP_d.npy')
####
DLPFC_control_d = np.load('control_DLPFC_d.npy')
DLPFC_0_d = np.load('0_DLPFC_d.npy')
DLPFC_100_d = np.load('100_DLPFC_d.npy')
DLPFC_200_d = np.load('200_DLPFC_d.npy')
DLPFC_800_d = np.load('800_DLPFC_d.npy')

### Response
LIP_control_r = np.load('control_LIP_r.npy')
LIP_0_r = np.load('0_LIP_r.npy')
LIP_100_r = np.load('100_LIP_r.npy')
LIP_200_r = np.load('200_LIP_r.npy')
LIP_800_r = np.load('800_LIP_r.npy')
####
DLPFC_control_r = np.load('control_DLPFC_r.npy')
DLPFC_0_r = np.load('0_DLPFC_r.npy')
DLPFC_100_r = np.load('100_DLPFC_r.npy')
DLPFC_200_r = np.load('200_DLPFC_r.npy')
DLPFC_800_r = np.load('800_DLPFC_r.npy')


### Shuffle Cross-decoder of Target

#### The labels are rolled N+1 100 times
#### One value for time and condition
#### For each condition, I make 100 decodings of the target in all the time points.
#### I end up with 100 vectors of 1x22 for each condition
#### Each bin contains 100 dots

#### Function to use

In [8]:

def decoding_spikes_angles_err(df, size_test=0.1):
    #### Input : dataframe with three columns: (spikes, behaviour and beuron label)
    ###Take off nans
    df = df.loc[df.iloc[:,0]<9999]
    df.columns=['spikes', 'beh', 'neuron']
    neuron_means=[]
    ### Train and test on the same neuron --> append the error of each neuron
    pop_err=[]
    
    for Neur in df.neuron.unique():
        ### same neuron 5 times --> mean value
        neur_err=[]
        for rep in range(0,5):
            spikes_train, spikes_test, beh_train, beh_test = train_test_split(df.groupby('neuron').get_group(Neur)['spikes'],
                                                                              df.groupby('neuron').get_group(Neur)['beh'],
                                                                              test_size=size_test)  

            ######## Trainning #########
            ## X matrix (intercept and spikes)
            X = np.column_stack([np.ones(np.shape(spikes_train)[0]),spikes_train])
            ## Y (sinus and cos)
            sinus =np.sin([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
            cosinus = np.cos([np.radians(np.array(beh_train)[i]) for i in range(0, len(beh_train))])
            Y = np.column_stack([cosinus, sinus])
            ### one OLS for sin and cos: output: beta of intercetp and bea of spikes (two B intercepts and 2 B for spikes )
            model = sm.OLS(Y, X)
            ##train the model
            fit=model.fit()

            ######### Testing ###########
            X = np.column_stack([np.ones(np.shape(spikes_test)[0]),spikes_test])
            p = fit.predict(X)
            x = p[:,0]
            y = p[:,1]
            #####
            ##### Error --> take the resulting vector in sin/cos space
            ### from sin and cos get the angle (-pi, pi)
            #pred_angle = [ np.degrees(np.arctan2(y[i], x[i]) + np.pi) for i in range(0, len(y))]
            pred_angle = [ np.degrees(np.arctan2(y[i], x[i])) for i in range(0, len(y))]
            for i in range(0, len(pred_angle)):
                if pred_angle[i]<0:
                    pred_angle[i]=360+pred_angle[i]
            ##
            error=[ circdist(beh_test[i], pred_angle[i]) for i in range(0, len(pred_angle))]

            #low_value --> predicted positionns close to real
            neur_err.append(np.mean(error))
            ####
        
        #####       
        pop_err.append(np.mean(neur_err))
        ####
    
    return np.mean(pop_err)

## Decoding Distractor

In [ ]:
Conditions_titles = ['LIP_0_d', 'LIP_100_d', 'LIP_200_d', 'LIP_800_d', 'DLPFC_0_d', 'DLPFC_100_d', 'DLPFC_200_d', 'DLPFC_800_d']
Conditions = [LIP_0_d, LIP_100_d, LIP_200_d, LIP_800_d, DLPFC_0_d, DLPFC_100_d, DLPFC_200_d, DLPFC_800_d]

conditions_shuff=[]
times_shuff=[[] for N in range(0, 100)]


for cond in range(0,8):
    times_shuff=[[] for N in range(0, 100)]
    for N in range(0,100):
        print('d', cond, N)
        shuff=Conditions[cond].copy()
        a = np.roll(shuff[29].values, N+1)
        shuff[28]=a
        for time in range(0, 22):
            df = shuff[[time, 29, 33]]
            times_shuff[N].append(decoding_spikes_angles_err(df))
    
    ##
    conditions_shuff.append(times_shuff)
    T = pd.DataFrame(conditions_shuff[cond])
    sns.pointplot(data=T, label=Conditions_titles[cond])
    
    
## it was done in two times (0-18, 18-22)
#os.chdir('C:\\Users\\David\\Dropbox\\IDIBAPS\\Distractor_neurons\\Shuffle')
os.chdir('/home/david/Desktop/IDIBAPS/Gottlib/Shuffle')

np.save('Shuffle_conditions_d', np.array(conditions_shuff))



## Decoding Target

In [ ]:
Conditions_titles = ['LIP_0', 'LIP_100', 'LIP_200', 'LIP_800', 'DLPFC_0', 'DLPFC_100', 'DLPFC_200', 'DLPFC_800']
Conditions = [LIP_0, LIP_100, LIP_200, LIP_800, DLPFC_0, DLPFC_100, DLPFC_200, DLPFC_800]

conditions_shuff=[]
times_shuff=[[] for N in range(0, 100)]


for cond in range(0,8):    
    times_shuff=[[] for N in range(0, 100)]
    for N in range(0,100):
        print('t', cond, N)
        shuff=Conditions[cond].copy()
        a = np.roll(shuff[28].values, N+1)
        shuff[28]=a
        for time in range(0, 22):
            df = shuff[[time,28, 33]]
            times_shuff[N].append(decoding_spikes_angles_err(df))
    
    ##
    conditions_shuff.append(times_shuff)
    T = pd.DataFrame(conditions_shuff[cond])
    sns.pointplot(data=T, label=Conditions_titles[cond])
    
    
## it was done in two times (0-18, 18-22)
#os.chdir('C:\\Users\\David\\Dropbox\\IDIBAPS\\Distractor_neurons\\Shuffle')
os.chdir('/home/david/Desktop/IDIBAPS/Gottlib/Shuffle')

np.save('Shuffle_conditions', np.array(conditions_shuff))


In [ ]:
##plot target
shuffle_conditions = np.load('Shuffle_conditions.npy')
Colors=['b', 'y', 'g', 'k',   'r', 'cyan', 'orange', 'grey']
print(Conditions_titles)

for i in range(0,8):
    sns.pointplot(data=shuffle_conditions[i], color=Colors[i], label=Conditions_titles[i], legend=True)

In [ ]:
##plot distractor
shuffle_conditions_d = np.load('Shuffle_conditions_d.npy')
Colors=['b', 'y', 'g', 'k',   'r', 'cyan', 'orange', 'grey']
print(Conditions_titles)

for i in range(0,8):
    sns.pointplot(data=shuffle_conditions_d[i], color=Colors[i], label=Conditions_titles[i], legend=True)